In [18]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
import random
import pickle

# Tải dữ liệu
dulieuLoad = pd.read_csv("Car_Insurance_balanced.csv")

# Tách các cột đặc trưng (X) và cột nhãn (Y)
dulieu_X_temp = dulieuLoad.iloc[:, :-1]
dulieu_Y_temp = dulieuLoad.iloc[:, -1]

# Chuyển đổi sang kiểu dữ liệu float64 (nếu cần)
dulieu_X = dulieu_X_temp.astype(np.float64)
dulieu_Y = dulieu_Y_temp.astype(np.float64)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
dulieu_X = scaler.fit_transform(dulieu_X)

# Lưu scaler đã huấn luyện
pickle.dump(scaler, open('scaler.pkl', 'wb'))

# Khởi tạo biến lưu mô hình tốt nhất và độ chính xác tốt nhất
best_model = None
best_val_accuracy = 0

# Danh sách để lưu các chỉ số của từng lần lặp
accuracy_scores = []
recall_scores = []
precision_scores = []
f1_scores = []

# Huấn luyện mô hình 10 lần
for each in range(10):
    combined = list(zip(dulieu_X, dulieu_Y))
    random.shuffle(combined)
    dulieu_X_shuffled, dulieu_Y_shuffled = zip(*combined)

    X_Train, X_Test, Y_Train, Y_Test = train_test_split(dulieu_X_shuffled, dulieu_Y_shuffled, test_size=1/3, random_state=100)

    # Tạo và huấn luyện mô hình Logistic Regression
    model = LogisticRegression(max_iter=1000, random_state=100)
    model.fit(X_Train, Y_Train)

    # Dự đoán trên tập kiểm tra
    Y_Test_Pred = model.predict(X_Test)

    # Tính toán các chỉ số
    val_accuracy = accuracy_score(Y_Test, Y_Test_Pred)
    model_recall = recall_score(Y_Test, Y_Test_Pred, average="binary")
    model_precision = precision_score(Y_Test, Y_Test_Pred, average="binary")
    model_f1 = f1_score(Y_Test, Y_Test_Pred, average="binary")

    # Lưu các chỉ số vào danh sách
    accuracy_scores.append(val_accuracy)
    recall_scores.append(model_recall)
    precision_scores.append(model_precision)
    f1_scores.append(model_f1)

    # Lưu mô hình tốt nhất dựa trên độ chính xác
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = model
        with open('best_logistic_regression_model.pkl', 'wb') as f:
            pickle.dump(best_model, f)

# Tính trung bình các chỉ số
avg_accuracy = np.mean(accuracy_scores)
avg_recall = np.mean(recall_scores)
avg_precision = np.mean(precision_scores)
avg_f1 = np.mean(f1_scores)

# In ra các chỉ số trung bình với 2 chữ số sau dấu phẩy
print("Trung bình sau 10 lần lặp của giải thuật Logistic Regression:")
print("Average Accuracy:", round(avg_accuracy, 2))
print("Average Recall:", round(avg_recall, 2))
print("Average Precision:", round(avg_precision, 2))
print("Average F1-Score:", round(avg_f1, 2))


Trung bình sau 10 lần lặp của giải thuật Logistic Regression:
Average Accuracy: 0.83
Average Recall: 0.85
Average Precision: 0.82
Average F1-Score: 0.83


In [22]:
import numpy as np
import pandas as pd
import pickle  # Cần thêm để tải scaler đã lưu

# Tải mô hình tốt nhất
with open('best_logistic_regression_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

# Tải scaler đã lưu
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Dữ liệu đầu vào để dự đoán
input_data = np.array([
    [3, 0, 0, 0, 0, 2, 0.629027313918201, 1, 0, 0, 1, 10238, 12000.0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 1, 0.357757117018462, 0, 1, 0, 0, 10238, 16000.0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 1, 0.357757117018462, 0, 1, 0, 0, 10238, 16000.0, 0, 0, 0, 0],
])

# Chuẩn hóa dữ liệu đầu vào bằng scaler đã tải
input_data_normalized = scaler.transform(input_data)

# Dự đoán nhãn cho dữ liệu đầu vào
predicted_label = best_model.predict(input_data_normalized)

# In ra kết quả dự đoán
for i, prediction in enumerate(predicted_label):
    print(f"Dự đoán nhãn cho dữ liệu đầu vào {i + 1}: {prediction}")


Dự đoán nhãn cho dữ liệu đầu vào 1: 0.0
Dự đoán nhãn cho dữ liệu đầu vào 2: 1.0
Dự đoán nhãn cho dữ liệu đầu vào 3: 1.0


f:\NamTu\HK1\KhaiKhoangDuLieu\Project_Nhom5_STT21\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [23]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier  # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
import random
import joblib

# Tải dữ liệu
dulieuLoad = pd.read_csv("Car_Insurance_balanced.csv")

# Tách các cột đặc trưng (X) và cột nhãn (Y)
dulieu_X_temp = dulieuLoad.iloc[:, :-1]
dulieu_Y_temp = dulieuLoad.iloc[:, -1]

# Chuyển đổi sang kiểu dữ liệu float64 (nếu cần)
dulieu_X = dulieu_X_temp.astype(np.float64)
dulieu_Y = dulieu_Y_temp.astype(np.float64)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
dulieu_X = scaler.fit_transform(dulieu_X)


# Khởi tạo biến lưu mô hình tốt nhất và độ chính xác tốt nhất
best_model = None
best_val_accuracy = 0

# Danh sách để lưu các chỉ số của từng lần lặp
accuracy_scores = []
recall_scores = []
precision_scores = []
f1_scores = []

# Huấn luyện mô hình 10 lần
for each in range(10):
    combined = list(zip(dulieu_X, dulieu_Y))
    random.shuffle(combined)
    dulieu_X_shuffled, dulieu_Y_shuffled = zip(*combined)

    X_Train, X_Test, Y_Train, Y_Test = train_test_split(dulieu_X_shuffled, dulieu_Y_shuffled, test_size=1/3, random_state=100)

    # Tạo và huấn luyện mô hình Decision Tree
    model = DecisionTreeClassifier(random_state=100)  # Sử dụng Decision Tree Classifier
    model.fit(X_Train, Y_Train)

    # Dự đoán trên tập kiểm tra
    Y_Test_Pred = model.predict(X_Test)

    # Tính toán các chỉ số
    val_accuracy = accuracy_score(Y_Test, Y_Test_Pred)
    model_recall = recall_score(Y_Test, Y_Test_Pred, average="binary")
    model_precision = precision_score(Y_Test, Y_Test_Pred, average="binary")
    model_f1 = f1_score(Y_Test, Y_Test_Pred, average="binary")

    # Lưu các chỉ số vào danh sách
    accuracy_scores.append(val_accuracy)
    recall_scores.append(model_recall)
    precision_scores.append(model_precision)
    f1_scores.append(model_f1)

    # Lưu mô hình tốt nhất dựa trên độ chính xác
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = model
        joblib.dump(best_model, 'best_decision_tree_model.pkl')  # Lưu mô hình vào file

# Tính trung bình các chỉ số
avg_accuracy = np.mean(accuracy_scores)
avg_recall = np.mean(recall_scores)
avg_precision = np.mean(precision_scores)
avg_f1 = np.mean(f1_scores)

# In ra các chỉ số trung bình với 2 chữ số sau dấu phẩy
print("Trung bình sau 10 lần lặp của giải thuật Decision Tree:")
print("Average Accuracy:", round(avg_accuracy, 2))
print("Average Recall:", round(avg_recall, 2))
print("Average Precision:", round(avg_precision, 2))
print("Average F1-Score:", round(avg_f1, 2))

Trung bình sau 10 lần lặp của giải thuật Decision Tree:
Average Accuracy: 0.77
Average Recall: 0.76
Average Precision: 0.77
Average F1-Score: 0.77


In [38]:
import numpy as np
import pandas as pd
import joblib  # Thư viện để tải mô hình đã lưu
from sklearn.preprocessing import StandardScaler
# Tải mô hình tốt nhất
best_model = joblib.load('best_decision_tree_model.pkl')
# Dữ liệu đầu vào để dự đoán
input_data = np.array([
    [3, 0, 0, 0, 0, 2, 0.629027313918201, 1.0, 0, 0.0, 1.0, 10238.0, 12000.0, 0, 0.0, 0, 0],
    [0, 1, 0, 0, 1, 1, 0.357757117018462, 0.0, 1, 0.0, 0.0, 10238.0, 16000.0, 0, 0.0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0.493145785218198, 1.0, 1, 0.0, 0.0, 10238.0, 11000.0, 0, 0.0, 0, 0],
    [0, 1, 0, 0, 2, 3, 0.206012850732455, 1.0, 1, 0.0, 1.0, 32765.0, 11000.0, 0, 0.0, 0, 0],
    [1, 1, 0, 1, 1, 3, 0.388365888157218, 1.0, 1, 0.0, 0.0, 32765.0, 12000.0, 0, 2.0, 0, 1]
])

scaler = StandardScaler()
# Chuẩn hóa dữ liệu đầu vào
input_data_normalized = scaler.fit_transform(input_data)
# Dự đoán nhãn cho dữ liệu đầu vào
predicted_label = best_model.predict(input_data_normalized)
# In ra kết quả dự đoán
for i, prediction in enumerate(predicted_label):
    print(f"Dự đoán nhãn cho dữ liệu đầu vào {i+1}: {prediction}")

Dự đoán nhãn cho dữ liệu đầu vào 1: 0.0
Dự đoán nhãn cho dữ liệu đầu vào 2: 1.0
Dự đoán nhãn cho dữ liệu đầu vào 3: 0.0
Dự đoán nhãn cho dữ liệu đầu vào 4: 0.0
Dự đoán nhãn cho dữ liệu đầu vào 5: 1.0


In [37]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
import random
import joblib

# Tải dữ liệu
dulieuLoad = pd.read_csv("Car_Insurance_balanced.csv")

# Tách các cột đặc trưng (X) và cột nhãn (Y)
dulieu_X_temp = dulieuLoad.iloc[:, :-1]
dulieu_Y_temp = dulieuLoad.iloc[:, -1]

# Chuyển đổi sang kiểu dữ liệu float64 (nếu cần)
dulieu_X = dulieu_X_temp.astype(np.float64)
dulieu_Y = dulieu_Y_temp.astype(np.float64)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
dulieu_X = scaler.fit_transform(dulieu_X)

# Khởi tạo biến lưu mô hình tốt nhất và độ chính xác tốt nhất
best_model = None
best_val_accuracy = 0

# Danh sách để lưu các chỉ số của từng lần lặp
accuracy_scores = []
recall_scores = []
precision_scores = []
f1_scores = []

# Huấn luyện mô hình 10 lần
for each in range(10):
    combined = list(zip(dulieu_X, dulieu_Y))
    random.shuffle(combined)
    dulieu_X_shuffled, dulieu_Y_shuffled = zip(*combined)

    X_Train, X_Test, Y_Train, Y_Test = train_test_split(dulieu_X_shuffled, dulieu_Y_shuffled, test_size=1/3, random_state=100)

    # Tạo và huấn luyện mô hình Gaussian Naive Bayes
    model = GaussianNB()
    model.fit(X_Train, Y_Train)

    # Dự đoán trên tập kiểm tra
    Y_Test_Pred = model.predict(X_Test)

    # Tính toán các chỉ số
    val_accuracy = accuracy_score(Y_Test, Y_Test_Pred)
    model_recall = recall_score(Y_Test, Y_Test_Pred, average="binary")
    model_precision = precision_score(Y_Test, Y_Test_Pred, average="binary")
    model_f1 = f1_score(Y_Test, Y_Test_Pred, average="binary")

    # Lưu các chỉ số vào danh sách
    accuracy_scores.append(val_accuracy)
    recall_scores.append(model_recall)
    precision_scores.append(model_precision)
    f1_scores.append(model_f1)

    # Lưu mô hình tốt nhất dựa trên độ chính xác
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = model
        joblib.dump(best_model, 'best_gaussian_nb_model.pkl')

# Tính trung bình các chỉ số
avg_accuracy = np.mean(accuracy_scores)
avg_recall = np.mean(recall_scores)
avg_precision = np.mean(precision_scores)
avg_f1 = np.mean(f1_scores)

# In ra các chỉ số trung bình với 2 chữ số sau dấu phẩy
print("Trung bình sau 10 lần lặp của giải thuật Gaussian Naive Bayes:")
print("Average Accuracy:", round(avg_accuracy, 2))
print("Average Recall:", round(avg_recall, 2))
print("Average Precision:", round(avg_precision, 2))
print("Average F1-Score:", round(avg_f1, 2))


Trung bình sau 10 lần lặp của giải thuật Gaussian Naive Bayes:
Average Accuracy: 0.78
Average Recall: 0.85
Average Precision: 0.74
Average F1-Score: 0.79


In [36]:
import numpy as np
import pandas as pd
import joblib  # Thư viện để tải mô hình đã lưu
from sklearn.preprocessing import StandardScaler
# Tải mô hình tốt nhất
best_model = joblib.load('best_gaussian_nb_model.pkl')
# Dữ liệu đầu vào để dự đoán
input_data = np.array([
    [3, 0, 0, 0, 0, 2, 0.629027313918201, 1.0, 0, 0.0, 1.0, 10238.0, 12000.0, 0, 0.0, 0, 0],
    [0, 1, 0, 0, 1, 1, 0.357757117018462, 0.0, 1, 0.0, 0.0, 10238.0, 16000.0, 0, 0.0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0.493145785218198, 1.0, 1, 0.0, 0.0, 10238.0, 11000.0, 0, 0.0, 0, 0],
    [0, 1, 0, 0, 2, 3, 0.206012850732455, 1.0, 1, 0.0, 1.0, 32765.0, 11000.0, 0, 0.0, 0, 0],
    [1, 1, 0, 1, 1, 3, 0.388365888157218, 1.0, 1, 0.0, 0.0, 32765.0, 12000.0, 0, 2.0, 0, 1]
])

scaler = StandardScaler()
# Chuẩn hóa dữ liệu đầu vào
input_data_normalized = scaler.fit_transform(input_data)
# Dự đoán nhãn cho dữ liệu đầu vào
predicted_label = best_model.predict(input_data_normalized)
# In ra kết quả dự đoán
for i, prediction in enumerate(predicted_label):
    print(f"Dự đoán nhãn cho dữ liệu đầu vào {i+1}: {prediction}")

Dự đoán nhãn cho dữ liệu đầu vào 1: 0.0
Dự đoán nhãn cho dữ liệu đầu vào 2: 1.0
Dự đoán nhãn cho dữ liệu đầu vào 3: 1.0
Dự đoán nhãn cho dữ liệu đầu vào 4: 1.0
Dự đoán nhãn cho dữ liệu đầu vào 5: 0.0
